## Part A. Build a baseline model

In [1]:
import pandas as pd
import numpy as np

### A.1 Data preparation

Let's download the data and read it into a pandas dataframe.

In [2]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


Let's check the data

In [3]:
# check the size
concrete_data.shape

(1030, 9)

In [4]:
# calculate basic statistical parameters
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
#check for nulls
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

Let's split data into predictors and target

In [6]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [7]:
# check out the predictors
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
# check out the target
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Let's save the number of predictors to n_cols.

In [9]:
n_cols = predictors.shape[1] # number of predictors

### A.2 Build the model

Let's import keras

In [10]:
import keras

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

Let's import the rest of the packages from the Keras library that we will need to build our regressoin model.

In [11]:
from keras.models import Sequential
from keras.layers import Dense

Let's define a function that defines a regression model with one hidden layer of 10 nodes, and a ReLU activation function.

In [12]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

### A.3 Train and evaluate the model 50 times

Let's define a function that trains and evaluate a model any chosen number of times (n).

In [13]:
#import needed functions from sklearn

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/utils/validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'


In [14]:
# define function

def run_model_n_times(n, chosen_model, X, y, epochs_num):
    #create empy list for mean squared errors to come
    mean_squared_errors = []
    
    for i in range(n):
    
        #create train and test datasets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 100)
    
        #build model
        model = chosen_model
    
        #fit model
        model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=epochs_num, verbose=2)
        
        #evaluate model
        scores = model.evaluate(X_test, y_test, verbose=0)
        
        mean_squared_errors.append(scores)
    
    return(mean_squared_errors)
        


In [15]:
mean_squared_errors = run_model_n_times(50, regression_model(), predictors, target, 50)







Train on 721 samples, validate on 309 samples
Epoch 1/50


2022-10-14 19:24:10.579307: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
2022-10-14 19:24:10.585315: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2593900000 Hz
2022-10-14 19:24:10.586214: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55b682cbe390 executing computations on platform Host. Devices:
2022-10-14 19:24:10.586286: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2022-10-14 19:24:10.665677: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line fl

 - 0s - loss: 38771.4262 - val_loss: 20680.3878
Epoch 2/50
 - 0s - loss: 11391.2379 - val_loss: 5775.3348
Epoch 3/50
 - 0s - loss: 4418.0965 - val_loss: 3785.0648
Epoch 4/50
 - 0s - loss: 3759.7496 - val_loss: 3526.7338
Epoch 5/50
 - 0s - loss: 3519.3066 - val_loss: 3309.5087
Epoch 6/50
 - 0s - loss: 3278.3580 - val_loss: 3084.1102
Epoch 7/50
 - 0s - loss: 3049.1866 - val_loss: 2873.3540
Epoch 8/50
 - 0s - loss: 2831.0572 - val_loss: 2670.1766
Epoch 9/50
 - 0s - loss: 2627.3187 - val_loss: 2479.2880
Epoch 10/50
 - 0s - loss: 2439.3104 - val_loss: 2311.2332
Epoch 11/50
 - 0s - loss: 2264.3841 - val_loss: 2144.6508
Epoch 12/50
 - 0s - loss: 2103.5332 - val_loss: 1999.2152
Epoch 13/50
 - 0s - loss: 1955.1019 - val_loss: 1855.1726
Epoch 14/50
 - 0s - loss: 1815.5855 - val_loss: 1735.1960
Epoch 15/50
 - 0s - loss: 1695.3369 - val_loss: 1623.9329
Epoch 16/50
 - 0s - loss: 1576.7471 - val_loss: 1516.1570
Epoch 17/50
 - 0s - loss: 1475.6815 - val_loss: 1420.5650
Epoch 18/50
 - 0s - loss: 1377.

In [16]:
print(mean_squared_errors)

[360.30625387148564, 120.71983488014986, 111.75763166606619, 111.3351506106676, 113.53976008344236, 115.04051366515915, 117.6178978484811, 113.01112321279581, 122.49795947028595, 111.76159645747212, 117.27369976352334, 117.40972604103459, 111.38875226758445, 111.14442525326626, 111.89525898066154, 111.86568611416617, 112.58526685399917, 113.84588361326544, 113.75204230743705, 111.3556443248366, 111.62098955567987, 113.7320002583624, 112.94874380167248, 111.9656086400103, 111.85702346752376, 112.03010650437241, 112.14958615596241, 113.84746761383748, 120.68541754565193, 118.73680341899588, 114.24929078728636, 110.91640020574181, 112.01843360481139, 119.71717686329073, 112.52424666025107, 113.08504493491164, 118.10084168270568, 111.38911516998192, 110.82942239211987, 110.48918695048607, 111.22595024726152, 114.25728668904227, 111.97969610714217, 112.35756392000563, 112.5985466670064, 113.57667835322013, 112.60269147755645, 111.43296421384349, 112.6882262739163, 112.8745249282195]


### Final answear to Part A

In [17]:
import statistics

In [20]:
print("Standard Deviation of the mean squared errors is % s ."% (statistics.stdev(mean_squared_errors)))
print("Mean of the mean squared errors is % s ." % (statistics.mean(mean_squared_errors))) 

Standard Deviation of the mean squared errors is 35.0044889689544 .
Mean of the mean squared errors is 118.571822847533 .


## Part B Normalize the data

Let's normalize the data by substracting the mean and dividing by the standard deviation.

In [21]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Let's fit and evaluate the model from part A again (50 times).

In [ ]:
mean_squared_errors_B = run_model_n_times(50, regression_model(), predictors_norm, target, 50)

Train on 721 samples, validate on 309 samples
Epoch 1/50
 - 0s - loss: 1521.6196 - val_loss: 1545.3518
Epoch 2/50
 - 0s - loss: 1506.8734 - val_loss: 1530.6846
Epoch 3/50
 - 0s - loss: 1492.2085 - val_loss: 1515.6400
Epoch 4/50
 - 0s - loss: 1477.0946 - val_loss: 1500.4026
Epoch 5/50
 - 0s - loss: 1461.6886 - val_loss: 1484.5448
Epoch 6/50
 - 0s - loss: 1445.4921 - val_loss: 1467.8987
Epoch 7/50
 - 0s - loss: 1428.3474 - val_loss: 1450.0995
Epoch 8/50
 - 0s - loss: 1410.1077 - val_loss: 1430.8418
Epoch 9/50
 - 0s - loss: 1390.7624 - val_loss: 1410.2400
Epoch 10/50
 - 0s - loss: 1369.8454 - val_loss: 1388.7461
Epoch 11/50
 - 0s - loss: 1347.6761 - val_loss: 1365.6138
Epoch 12/50
 - 0s - loss: 1324.0122 - val_loss: 1340.6031
Epoch 13/50
 - 0s - loss: 1298.6216 - val_loss: 1314.2533
Epoch 14/50
 - 0s - loss: 1272.0632 - val_loss: 1286.5276
Epoch 15/50
 - 0s - loss: 1243.9588 - val_loss: 1257.2234
Epoch 16/50
 - 0s - loss: 1214.1751 - val_loss: 1226.9122
Epoch 17/50
 - 0s - loss: 1183.3603

### Final answear to Part B

In [24]:
print("Standard Deviation of the mean squared errors in Part B is % s as compared to % s in Part A."% (statistics.stdev(mean_squared_errors_B), statistics.stdev(mean_squared_errors)))
print("Mean of the mean squared errors in Part B is % s as compared to % s in Part A." % (statistics.mean(mean_squared_errors_B), statistics.mean(mean_squared_errors))) 

Standard Deviation of the mean squared errors in Part B is 41.75280439597244 as compared to 35.0044889689544 in Part A.
Mean of the mean squared errors in Part B is 62.672951418435304 as compared to 118.571822847533 in Part A.


## Part C Increate the number of epochs

Let's fit and evaluate the model from part B again (50 times), but use 100 epochs this time for training. 

In [26]:
mean_squared_errors_C = run_model_n_times(50, regression_model(), predictors_norm, target, 100)

Train on 721 samples, validate on 309 samples
Epoch 1/100
 - 0s - loss: 1536.9802 - val_loss: 1560.1874
Epoch 2/100
 - 0s - loss: 1522.0789 - val_loss: 1545.3556
Epoch 3/100
 - 0s - loss: 1506.9890 - val_loss: 1529.9987
Epoch 4/100
 - 0s - loss: 1491.3036 - val_loss: 1514.0860
Epoch 5/100
 - 0s - loss: 1475.0426 - val_loss: 1497.4518
Epoch 6/100
 - 0s - loss: 1458.0225 - val_loss: 1480.0846
Epoch 7/100
 - 0s - loss: 1439.8483 - val_loss: 1462.2571
Epoch 8/100
 - 0s - loss: 1420.7386 - val_loss: 1443.0097
Epoch 9/100
 - 0s - loss: 1400.6467 - val_loss: 1422.7184
Epoch 10/100
 - 0s - loss: 1379.3107 - val_loss: 1401.4156
Epoch 11/100
 - 0s - loss: 1356.8641 - val_loss: 1379.2488
Epoch 12/100
 - 0s - loss: 1333.5300 - val_loss: 1355.1654
Epoch 13/100
 - 0s - loss: 1308.9180 - val_loss: 1330.7470
Epoch 14/100
 - 0s - loss: 1283.3669 - val_loss: 1305.0162
Epoch 15/100
 - 0s - loss: 1257.0952 - val_loss: 1278.4171
Epoch 16/100
 - 0s - loss: 1229.6407 - val_loss: 1251.0171
Epoch 17/100
 - 0s 

### Final answear to Part C

In [28]:
print("Mean of the mean squared errors in Part C is % s as compared to % s in Part B." % (statistics.mean(mean_squared_errors_C), statistics.mean(mean_squared_errors_B))) 

Mean of the mean squared errors in Part C is 45.611690445440104 as compared to 62.672951418435304 in Part B.


## Part D Increase the number of hidden layers

Let's define a new model with 3 hidden layers, each of 10 nodes and ReLU activation function.

In [30]:
# define new regression model
def regression_model_D():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

Let's fit and evaluate the new model (50 times).

In [31]:
mean_squared_errors_D = run_model_n_times(50, regression_model_D(), predictors_norm, target, 50)

Train on 721 samples, validate on 309 samples
Epoch 1/50
 - 0s - loss: 1545.2399 - val_loss: 1567.0667
Epoch 2/50
 - 0s - loss: 1521.2457 - val_loss: 1538.0079
Epoch 3/50
 - 0s - loss: 1476.0238 - val_loss: 1484.8135
Epoch 4/50
 - 0s - loss: 1402.8780 - val_loss: 1403.1635
Epoch 5/50
 - 0s - loss: 1296.3921 - val_loss: 1286.4031
Epoch 6/50
 - 0s - loss: 1151.1397 - val_loss: 1130.8331
Epoch 7/50
 - 0s - loss: 967.3914 - val_loss: 944.6915
Epoch 8/50
 - 0s - loss: 767.0249 - val_loss: 748.8745
Epoch 9/50
 - 0s - loss: 579.5464 - val_loss: 579.4745
Epoch 10/50
 - 0s - loss: 440.1552 - val_loss: 449.1775
Epoch 11/50
 - 0s - loss: 347.9058 - val_loss: 361.0493
Epoch 12/50
 - 0s - loss: 289.9028 - val_loss: 307.2120
Epoch 13/50
 - 0s - loss: 255.2042 - val_loss: 275.5514
Epoch 14/50
 - 0s - loss: 233.5981 - val_loss: 252.2180
Epoch 15/50
 - 0s - loss: 219.4663 - val_loss: 236.9009
Epoch 16/50
 - 0s - loss: 209.4956 - val_loss: 224.1446
Epoch 17/50
 - 0s - loss: 202.2083 - val_loss: 213.2172

### Final answear to Part D

In [32]:
print("Mean of the mean squared errors in Part D is % s as compared to % s in Part B." % (statistics.mean(mean_squared_errors_D), statistics.mean(mean_squared_errors_B))) 

Mean of the mean squared errors in Part D is 36.37361456725976 as compared to 62.672951418435304 in Part B.
